# 🔍 Building an AI-Powered Web Search Agent with OpenAI and Tavily 🚀

Hey there! Welcome to this exciting guide where we'll create something awesome - a smart search agent that combines the power of OpenAI's language models with Tavily's search capabilities! 🌟 

## 🎯 What We'll Build

We're going to create a super cool search agent that can:
1. 🌐 Search the web in real-time for accurate information
2. 🧠 Use OpenAI's powerful GPT models to understand and process search results
3. ⚡ Provide contextual and up-to-date responses to queries

## ✅ Prerequisites

Before we jump in, make sure you have these things ready:
- 🔑 An OpenAI API key
- 🎯 A Tavily API key (get one at tavily.com)

## 🎮 Part 1: Setting Up Our Environment

First things first - let's get our tools ready! We'll need to install the Tavily Python package to interact with their search API:


In [1]:
%pip install tavily-python


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



Now, let's set up our API keys (remember to keep these safe and never share them!):


In [2]:
TAVILY_API = "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"


## 🛠️ Part 2: Building Our Search Tools

Let's create the foundation of our search agent! We'll define a set of tools that our AI can use to search the web:

In [3]:
import json
from openai import OpenAI
from tavily import TavilyClient

In [4]:
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API)

# Search query
query = "Whats the current weather in Tokyo"
tavily.search(query, search_depth="basic")

{'query': 'Whats the current weather in Tokyo',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Weather in Tokyo',
   'url': 'https://www.weatherapi.com/',
   'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1739879705, 'localtime': '2025-02-18 20:55'}, 'current': {'last_updated_epoch': 1739879100, 'last_updated': '2025-02-18 20:45', 'temp_c': 6.3, 'temp_f': 43.3, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 338, 'wind_dir': 'NNW', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.13, 'precip_in': 0.01, 'humidity': 33, 'cloud': 75, 'feelslike_c': 3.3, 'feelslike_f': 38.0, 'windchill_c': 1.2, 'windchill_f': 34.2, 'heatindex_c': 4.6, 'heatindex_f': 40.3, 'dewpoint_c': -8.3, 'dewpoint_f': 17.0, 'vis_km': 1

In [5]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Search the web for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            },
        },
    },
]

In [6]:
def search_web(query):
    # Initialize Tavily
    tavily = TavilyClient(api_key=TAVILY_API)
    return "\n".join(result["content"] for result in tavily.search(query, search_depth="basic")["results"])

In [7]:
search_web("What is the current weather in Tokyo")

"{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1739880049, 'localtime': '2025-02-18 21:00'}, 'current': {'last_updated_epoch': 1739880000, 'last_updated': '2025-02-18 21:00', 'temp_c': 6.3, 'temp_f': 43.3, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 351, 'wind_dir': 'N', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.03, 'precip_in': 0.0, 'humidity': 39, 'cloud': 75, 'feelslike_c': 2.8, 'feelslike_f': 37.1, 'windchill_c': 0.8, 'windchill_f': 33.5, 'heatindex_c': 4.7, 'heatindex_f': 40.5, 'dewpoint_c': -9.5, 'dewpoint_f': 15.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 15.2, 'gust_kph': 24.4}}\nWEATHER IN TOKYO Weather forecast in Tokyo Current weather in Tokyo Weather archive in Tokyo Weather history in Tokyo Weather summary in T


## 🎓 Part 3: Creating Our AI Agent

Now comes the exciting part! Let's create our AI agent that can understand questions and use our search tools to find answers:


In [8]:
messages = [
    {"role": "user", "content": "What is the current weather in Tokyo"}
]

In [9]:
def invoke_model(messages):
    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [10]:
# Initialize the OpenAI client
client = OpenAI()

# Make a ChatGPT API call with tool calling
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    tools=TOOLS,
    messages=messages
)

response = completion.choices[0].message

# Parse the response to get the tool call arguments
if response.tool_calls:
    # Process each tool call
    for tool_call in response.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "search_web":
            search_results = search_web(tool_call_arguments["query"])
            messages.append({"role": "assistant", "content": f"Here are the search results: {search_results}"})
            print(invoke_model(messages))

else:
    # If there are no tool calls, return the response content
    print(response.content)

As of now, the weather in Tokyo is partly cloudy with a temperature of approximately 6.3°C (43.3°F). It feels like 2.8°C (37.1°F) due to wind chill. The wind is coming from the north at about 11.9 mph (19.1 kph), and the humidity level is around 39%. Overall, visibility is good at 10 km (6 miles).

If you need more detailed information or a forecast, let me know!
